This file takes the original dataset from Kaggle and converts the dataset into Python-readable values

In [22]:
import numpy as np
import pandas as pd
from pandas import DataFrame
import math
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import date
from tabulate import tabulate
import sys
import copy
sys.path.append('C:\\Users\\chery\\Documents\\SeattlePolice911Response')
from utils.baseML import BaseML
from datetime import datetime
from sklearn import preprocessing

Read in CSV

In [23]:
spir_original = pd.read_csv("Seattle_Police_Department_911_Incident_Response.csv")

C:\Users\chery\AppData\Local\Programs\Python\Python39\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [4]:
print(spir_original.columns)

Index(['CAD CDW ID', 'CAD Event Number', 'General Offense Number',
       'Event Clearance Code', 'Event Clearance Description',
       'Event Clearance SubGroup', 'Event Clearance Group',
       'Event Clearance Date', 'Hundred Block Location', 'District/Sector',
       'Zone/Beat', 'Census Tract', 'Longitude', 'Latitude',
       'Incident Location', 'Initial Type Description',
       'Initial Type Subgroup', 'Initial Type Group', 'At Scene Time'],
      dtype='object')


 Drop unused Columns

In [24]:
drop_columns= ['CAD CDW ID', 'CAD Event Number', 'General Offense Number', 'Hundred Block Location', 'District/Sector',
       'Zone/Beat', 'Longitude', 'Latitude',
       'Incident Location']

spir_drop_unused_col = spir_original.drop(labels=drop_columns, axis=1)

Create list of unique census tracts. This is used to pull data from the US Census API 5 year American Community Survey

In [13]:
census_tracts = spir_drop_unused_col["Census Tract"].unique()
census_tracts = pd.DataFrame(census_tracts, columns=["census_tracts"])
census_tracts.dropna(axis =0, inplace=True)
census_tracts.census_tracts=census_tracts.census_tracts.apply(format_tract)
census_tracts.census_tracts = pd.Series(census_tracts.census_tracts.unique())
census_tracts.dropna(axis=0, inplace=True)
census_tracts.to_csv("census_tracts.csv")

Drop rows with missing values

In [25]:
spir_dropna = spir_drop_unused_col.dropna(axis=0, how='any')

Fix date and time

In [93]:
class Seattle_Police_ML(BaseML):

    def dayDifference(self, fdate, ldate):
        delta = ldate - fdate
        return(delta)

    def transform_times(self):
        dt_format = '%m/%d/%Y %I:%M:%S %p'
        #self.df["at_scene_datetime"] = self.df.apply(lambda x: x.At_Scene_Date + "T" + x.At_Scene_Time, axis=1)
        #self.df['at_scene_temp'] = self.df.at_scene_datetime.apply(lambda x: datetime.fromisoformat(x))
        self.df['at_scene_time'] = self.df["At Scene Time"].apply(lambda x: datetime.strptime(x, dt_format))
        #self.df.Event_Clearance_Date.fillna("2021-05-13", inplace=True)
        #self.df.Event_Clearance_Time.fillna("00:00:00", inplace=True)
        
        #self.df[["Event_Clearance_Date", "Event_Clearance_Time"]] = self.df[["Event_Clearance_Date", "Event_Clearance_Time"]].astype(str)
        #print(spir.df.Event_Clearance_Date.dtype)
        
        #self.df["event_clear_datetime"] = self.df.apply(lambda x: x.Event_Clearance_Date + "T" + x.Event_Clearance_Time, axis=1)
        #self.df['event_clear_temp'] = self.df.event_clear_datetime.apply(lambda x: datetime.fromisoformat(x))
        self.df['event_clear_time']=self.df["Event Clearance Date"].apply(lambda x: datetime.strptime(x, dt_format))
        self.df['time_at_scene'] = self.df.apply(lambda x: pd.to_timedelta(self.dayDifference(x["at_scene_time"], x["event_clear_time"])), axis = 1)
        self.df.drop(["At Scene Time", "Event Clearance Date"], axis = 1, inplace = True)
        #self.df.drop(self.df[self.df['Schedule_to_appt_days'] < 0].index, inplace = True)
        #self.df.dropna(inplace=True)
        print(self.df.sample(3))

    def simplify_times(self):
        bins = (
            pd.Timedelta(minutes = 0),
            pd.Timedelta(minutes = 30),
            pd.Timedelta(minutes = 60),
            pd.Timedelta(hours = 4),
            pd.Timedelta(hours = 8),
            pd.Timedelta(hours = 12),
            pd.Timedelta(days = 1),
            pd.Timedelta(days = 36500)
        )

        labels = ['< 30min', '30-60min','1-4hrs', '4-8hrs','8-12hrs','12-24hrs', '24hrs+']

        return pd.cut(self.df['time_at_scene'], bins, labels = labels)

    def map_func(self,parameter_df, x):
        try:
            return parameter_df.loc[math.floor(x["Census Tract"]), str(datetime.strptime(x.at_scene_time, "%Y-%m-%d %H:%M:%S").year)]
        except KeyError:
            return None
            pass
            #print(math.floor(x["Census Tract"]))

    
    def add_census_data(self, census_filename, parameter_name):
        parameter_df = pd.read_csv(census_filename, index_col=1)
        print(self.df["Census Tract"].sample(3))
        self.df[parameter_name]=self.df.apply(lambda x: self.map_func(parameter_df, x), axis=1)
        
        
    def normalize_features(self):
        scaler = preprocessing.StandardScaler().fit(self.df)
        df_scaled = scaler.transform(self.df)
        self.df_normalized = pd.DataFrame(df_scaled, columns = self.df.columns, dtype= 'int64')


In [46]:
year=datetime.today().year
test=pd.DataFrame([[1,2,3],
            ['a','b','c']], columns=[2021, 2012, 2013])
print(test[year])

print(spir_final.df["Census Tract"].sample(3))


0    1
1    a
Name: 2021, dtype: object
1038734    4800.3047
892743     8002.2005
1072815    7700.1029
Name: Census Tract, dtype: float64


In [49]:
test_df= pd.read_csv("white_percent.csv", index_col=1)
print(test_df.loc[math.floor(1200.2000), '2017'])

0.631121868


In [9]:
print(spir_drop_unused_col['Event Clearance Date'].sample(3))


65275      10/01/2010 05:07:00 PM
1066012    03/23/2017 05:38:44 AM
49629      09/12/2010 01:41:00 PM
Name: Event Clearance Date, dtype: object


In [84]:
spir_final = Seattle_Police_ML()
spir_final.df = copy.deepcopy(spir_dropna)
spir_final.transform_times()
spir_final.simplify_times()


         Event Clearance Code Event Clearance Description  \
798918                  280.0           SUSPICIOUS PERSON   
1106900                 245.0          DISTURBANCE, OTHER   
688610                  460.0  TRAFFIC (MOVING) VIOLATION   

         Event Clearance SubGroup     Event Clearance Group  Census Tract  \
798918   SUSPICIOUS CIRCUMSTANCES  SUSPICIOUS CIRCUMSTANCES     7200.1042   
1106900              DISTURBANCES              DISTURBANCES     4600.1001   
688610      TRAFFIC RELATED CALLS     TRAFFIC RELATED CALLS     7300.1003   

                               Initial Type Description  \
798918   WEAPN-IP/JO-GUN,DEADLY WPN (NO THRT/ASLT/DIST)   
1106900                             NUISANCE - MISCHIEF   
688610          TRAFFIC STOP - OFFICER INITIATED ONVIEW   

              Initial Type Subgroup          Initial Type Group  \
798918                WEAPONS CALLS               WEAPONS CALLS   
1106900  MISCELLANEOUS MISDEMEANORS  MISCELLANEOUS MISDEMEANORS   
688610  

255          1-4hrs
2147         1-4hrs
2376         1-4hrs
2433         1-4hrs
2444         1-4hrs
             ...   
1433846      1-4hrs
1433847    30-60min
1433850      1-4hrs
1433851      1-4hrs
1433852    30-60min
Name: time_at_scene, Length: 394768, dtype: category
Categories (7, object): ['< 30min' < '30-60min' < '1-4hrs' < '4-8hrs' < '8-12hrs' < '12-24hrs' < '24hrs+']

In [85]:
spir_final.df.to_csv("SPIR_transformed_times.csv")

In [154]:
spir_final = Seattle_Police_ML("SPIR_transformed_times.csv")
print(spir_final.df["Census Tract"].sample(3))
spir_final.add_census_data("white_percent.csv","white")


48387     8001.2008
103       8100.2023
250201    7200.3000
Name: Census Tract, dtype: float64
89936     11402.1000
110294    11800.6018
131127     6800.2007
Name: Census Tract, dtype: float64


In [155]:
spir_final.add_census_data("total_pop_by_tract.csv","total_pop")
spir_final.add_census_data("male_percent.csv","male")
spir_final.add_census_data("notUScitizen_percent.csv","notUScitizen")
spir_final.add_census_data("asian_percent.csv","asian")
spir_final.add_census_data("black_percent.csv","black")
spir_final.add_census_data("native_percent.csv","native")
spir_final.add_census_data("other_race_percent.csv","other_race")
spir_final.add_census_data("two_races_percent.csv","two_races")
spir_final.add_census_data("some_college_percent.csv","some_college")
spir_final.add_census_data("bachelors_percent.csv","bachelors")
spir_final.add_census_data("grad_deg_percent.csv","grad_deg")
spir_final.add_census_data("under18MC_percent.csv","under18MC")
spir_final.add_census_data("under18MS_percent.csv","under18MS")
spir_final.add_census_data("under18FS_percent.csv","under18FS")
spir_final.add_census_data("under18T_percent.csv","under18T")
spir_final.add_census_data("income_by_tract.csv", "income")


210098     100.3004
166021    3300.3019
300818    9300.2005
Name: Census Tract, dtype: float64
172503     3000.3004
52521     10900.2089
141970     4900.5014
Name: Census Tract, dtype: float64
230919    9300.3099
245990    1000.2007
208133    7200.1063
Name: Census Tract, dtype: float64
335577    1200.4005
257065    6700.1001
14887     8400.2006
Name: Census Tract, dtype: float64
392797     402.2007
6085      9300.3161
228938    9200.1006
Name: Census Tract, dtype: float64
223716    8900.6007
84791     7300.2010
168597    5802.2012
Name: Census Tract, dtype: float64
353209    8100.2008
235345    9800.4020
254037    5200.3004
Name: Census Tract, dtype: float64
361283    7401.1006
126277    7100.1003
218072    8200.2000
Name: Census Tract, dtype: float64
227608    11101.1005
257166      800.1005
124931     5400.1018
Name: Census Tract, dtype: float64
24548     9300.2060
343342    4900.2008
167087    7000.3015
Name: Census Tract, dtype: float64
28010     7900.4006
292912    8500.3001
4797

In [156]:
spir_final.df.to_csv("SPIR_census_added.csv")


In [157]:
spir_final.df.dropna(axis=0, inplace=True)

In [158]:
spir_final.df.columns

Index(['Unnamed: 0', 'Event Clearance Code', 'Event Clearance Description',
       'Event Clearance SubGroup', 'Event Clearance Group', 'Census Tract',
       'Initial Type Description', 'Initial Type Subgroup',
       'Initial Type Group', 'at_scene_time', 'event_clear_time',
       'time_at_scene', 'white', 'total_pop', 'male', 'notUScitizen', 'asian',
       'black', 'native', 'other_race', 'two_races', 'some_college',
       'bachelors', 'grad_deg', 'under18MC', 'under18MS', 'under18FS',
       'under18T', 'income'],
      dtype='object')

In [100]:
print(spir_final.df['Initial Type Description'].value_counts())

SUSPICIOUS PERSON, VEHICLE OR INCIDENT           28090
THEFT (DOES NOT INCLUDE SHOPLIFT OR SVCS)        27716
DISTURBANCE, MISCELLANEOUS/OTHER                 25488
TRAFFIC STOP - OFFICER INITIATED ONVIEW          19858
BURG - RES (INCL UNOCC STRUCTURES ON PROP)       17027
                                                 ...  
NORAD - INCIDENTS INV AIRCRAFT (EXCL CRASHES)        2
ALARM - EQUIPMENT                                    2
NARCOTICS - VIOLATION OF SODA ORDER                  1
TRU - PURSE SNATCH                                   1
TRU - LURING                                         1
Name: Initial Type Description, Length: 169, dtype: int64


In [159]:
spir_final.df.drop(["Unnamed: 0"], axis=1, inplace=True)

In [142]:
print(spir_final.df.columns)

Index(['Event Clearance Code', 'Census Tract', 'Initial Type Description',
       'at_scene_time', 'event_clear_time', 'time_at_scene', 'white',
       'total_pop', 'male', 'notUScitizen', 'asian', 'black', 'native',
       'other_race', 'two_races', 'some_college', 'bachelors', 'grad_deg',
       'under18MC', 'under18MS', 'under18FS', 'under18T', 'income'],
      dtype='object')


In [160]:
spir_final.df['Initial Type Description'] = spir_final.df['Initial Type Description'].astype('category')
print(spir_final.df.dtypes)

Event Clearance Code            float64
Event Clearance Description      object
Event Clearance SubGroup         object
Event Clearance Group            object
Census Tract                    float64
Initial Type Description       category
Initial Type Subgroup            object
Initial Type Group               object
at_scene_time                    object
event_clear_time                 object
time_at_scene                    object
white                           float64
total_pop                       float64
male                            float64
notUScitizen                    float64
asian                           float64
black                           float64
native                          float64
other_race                      float64
two_races                       float64
some_college                    float64
bachelors                       float64
grad_deg                        float64
under18MC                       float64
under18MS                       float64


In [161]:
spir_final.df["initial_type_desc_cat"] = spir_final.df["Initial Type Description"].cat.codes
print(spir_final.df.head())

   Event Clearance Code         Event Clearance Description  \
0                 161.0                            TRESPASS   
1                  50.0  BURGLARY - RESIDENTIAL, UNOCCUPIED   
2                 430.0             MOTOR VEHICLE COLLISION   
3                  50.0  BURGLARY - RESIDENTIAL, UNOCCUPIED   
4                 430.0             MOTOR VEHICLE COLLISION   

  Event Clearance SubGroup                  Event Clearance Group  \
0                 TRESPASS                               TRESPASS   
1   RESIDENTIAL BURGLARIES                               BURGLARY   
2    TRAFFIC RELATED CALLS  MOTOR VEHICLE COLLISION INVESTIGATION   
3   RESIDENTIAL BURGLARIES                               BURGLARY   
4    TRAFFIC RELATED CALLS  MOTOR VEHICLE COLLISION INVESTIGATION   

   Census Tract                           Initial Type Description  \
0     6000.1011                                NUISANCE - MISCHIEF   
1    10300.2005         BURG - RES (INCL UNOCC STRUCTURES ON PROP)

In [162]:
spir_final.df.at_scene_time = spir_final.df.at_scene_time.astype('datetime64')
spir_final.df.event_clear_time = spir_final.df.event_clear_time.astype('datetime64')
spir_final.df.time_at_scene = pd.to_timedelta(spir_final.df.time_at_scene)

print(spir_final.df.time_at_scene.dtypes)

timedelta64[ns]


In [163]:
print(spir_final.df.head())

   Event Clearance Code         Event Clearance Description  \
0                 161.0                            TRESPASS   
1                  50.0  BURGLARY - RESIDENTIAL, UNOCCUPIED   
2                 430.0             MOTOR VEHICLE COLLISION   
3                  50.0  BURGLARY - RESIDENTIAL, UNOCCUPIED   
4                 430.0             MOTOR VEHICLE COLLISION   

  Event Clearance SubGroup                  Event Clearance Group  \
0                 TRESPASS                               TRESPASS   
1   RESIDENTIAL BURGLARIES                               BURGLARY   
2    TRAFFIC RELATED CALLS  MOTOR VEHICLE COLLISION INVESTIGATION   
3   RESIDENTIAL BURGLARIES                               BURGLARY   
4    TRAFFIC RELATED CALLS  MOTOR VEHICLE COLLISION INVESTIGATION   

   Census Tract                           Initial Type Description  \
0     6000.1011                                NUISANCE - MISCHIEF   
1    10300.2005         BURG - RES (INCL UNOCC STRUCTURES ON PROP)

In [164]:
noon = datetime(2021, 11, 12,hour = 12)
spir_final.df["at_scene_time_pm"] = spir_final.df.at_scene_time.apply(lambda x: 0 if x.hour < noon.hour else 1)
spir_final.df["event_clear_time_pm"] = spir_final.df.event_clear_time.apply(lambda x: 0 if x.hour < noon.hour else 1)


In [165]:
spir_final.df["at_scene_time_weekday"] = spir_final.df.at_scene_time.apply(datetime.weekday)
spir_final.df["event_clear_time_weekday"] = spir_final.df.event_clear_time.apply(datetime.weekday)

In [166]:
spir_final.df["at_scene_time_month"] = spir_final.df.at_scene_time.apply(lambda x: x.month)
spir_final.df["event_clear_time_month"] = spir_final.df.event_clear_time.apply(lambda x: x.month)

In [167]:
spir_final.df["at_scene_time_year"] = spir_final.df.at_scene_time.apply(lambda x: x.year)
spir_final.df["event_clear_time_year"] = spir_final.df.event_clear_time.apply(lambda x: x.year)

In [151]:
spir_final.df.sample(10)

,Event Clearance Code,Census Tract,Initial Type Description,at_scene_time,event_clear_time,time_at_scene,white,total_pop,male,notUScitizen,...,income,initial_type_desc_cat,at_scene_time_pm,event_clear_time_pm,at_scene_time_weekday,event_clear_time_weekday,at_scene_time_month,event_clear_time_month,at_scene_time_year,event_clear_time_year
359683,245.0,7500.1002,"HARAS - NO BIAS, THREATS OR MALICIOUSNESS",2015-04-11 16:07:00,2015-04-11 17:14:00,0 days 01:07:00,0.741231,7783.0,0.528845,0.077091,...,63419.0,60,1,1,5,5,4,4,2015,2015
250938,282.0,4100.6005,PROWLER - IP/JO,2017-06-29 02:18:13,2017-06-29 03:30:19,0 days 01:12:06,0.829293,7920.0,0.513510,0.096591,...,168415.0,107,0,0,3,3,6,6,2017,2017
374215,161.0,8100.1032,TRESPASS,2015-05-31 21:49:00,2015-05-31 22:39:00,0 days 00:50:00,0.653890,4435.0,0.564600,0.177903,...,75531.0,137,1,1,6,6,5,5,2015,2015
186259,65.0,8500.3001,THEFT (DOES NOT INCLUDE SHOPLIFT OR SVCS),2016-07-24 12:25:41,2016-07-24 15:18:57,0 days 02:53:16,0.534384,4377.0,0.635367,0.099155,...,32885.0,126,1,1,6,6,7,7,2016,2016
175826,280.0,5301.3002,THEFT (DOES NOT INCLUDE SHOPLIFT OR SVCS),2012-09-06 11:34:00,2012-09-06 12:30:00,0 days 00:56:00,0.587797,6851.0,0.472778,0.141877,...,13563.0,126,0,1,3,3,9,9,2012,2012
129739,64.0,4700.3021,SHOPLIFT - THEFT,2015-08-31 16:11:00,2015-08-31 18:16:00,0 days 02:05:00,0.835974,8395.0,0.497439,0.071114,...,72396.0,119,1,1,0,0,8,8,2015,2015
325941,50.0,9900.1015,BURG - RES (INCL UNOCC STRUCTURES ON PROP),2014-12-02 22:03:00,2014-12-03 00:04:00,0 days 02:01:00,0.718990,4950.0,0.512323,0.060404,...,70822.0,35,1,0,1,2,12,12,2014,2014
21521,430.0,10100.5035,"MOTOR VEHICLE COLLISION, HIT AND RUN",2016-04-02 09:36:19,2016-04-02 10:34:34,0 days 00:58:15,0.384884,7449.0,0.490670,0.066989,...,59193.0,81,0,0,5,5,4,4,2016,2016
5447,184.0,9200.2014,"NARCOTICS - VIOLATIONS (LOITER, USE, SELL, NARS)",2016-02-24 14:44:18,2016-02-24 17:00:48,0 days 02:16:30,0.514734,2613.0,0.642556,0.102947,...,26500.0,86,1,1,2,2,2,2,2016,2016
328633,363.0,1900.4003,MISSING - ADULT,2014-12-11 20:56:00,2014-12-11 22:54:00,0 days 01:58:00,0.734617,4518.0,0.496016,0.086764,...,61979.0,77,1,1,3,3,12,12,2014,2014


In [168]:
from datetime import timedelta
spir_final.df["time_at_scene_seconds"] = spir_final.df.time_at_scene.apply(timedelta.total_seconds)

In [169]:
spir_final.df.to_csv("SPIR_encoded_2021116.csv")

Isolate missing values